In [11]:
import pandas as pd
import numpy as np
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

We use the same data here that was used to train the LightGBM model

In [12]:
df = pd.read_parquet('../data/processed_data/yellow_23-24_data.parquet')
df.head()

,pickup_count,lon,lat,time_sin,time_cos,day_sin,day_cos
0,1.386294,-74.174002,40.69183,0.000000,1.000000,-0.974928,-0.222521
1,1.609438,-74.174002,40.69183,0.258819,0.965926,-0.974928,-0.222521
2,2.079442,-74.174002,40.69183,0.500000,0.866025,-0.974928,-0.222521
3,2.197225,-74.174002,40.69183,0.707107,0.707107,-0.974928,-0.222521
4,2.772589,-74.174002,40.69183,0.866025,0.500000,-0.974928,-0.222521


In [13]:
y = df['pickup_count']
X = df.drop(columns=['pickup_count'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 2, 32, log=True),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0)
    }
    
    rf = RandomForestRegressor(**params, random_state=42)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(rf, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
    
    return -np.mean(scores)

study = optuna.create_study(direction='minimize')  # We want to minimize MSE
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best parameters:", best_params)

best_rf = RandomForestRegressor(**best_params, random_state=42)
best_rf.fit(X_train, y_train)

[I 2024-12-09 16:28:48,197] A new study created in memory with name: no-name-aad99468-c083-4b8f-962b-825044c75b1a
[I 2024-12-09 16:29:30,054] Trial 0 finished with value: 0.7479169455254399 and parameters: {'n_estimators': 991, 'max_depth': 12, 'min_samples_split': 84, 'min_samples_leaf': 6, 'max_features': 0.4422486120870721}. Best is trial 0 with value: 0.7479169455254399.
[I 2024-12-09 16:30:08,657] Trial 1 finished with value: 0.1644008990458268 and parameters: {'n_estimators': 336, 'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 0.7768424530830121}. Best is trial 1 with value: 0.1644008990458268.
[I 2024-12-09 16:30:21,426] Trial 2 finished with value: 2.963690919723827 and parameters: {'n_estimators': 682, 'max_depth': 5, 'min_samples_split': 16, 'min_samples_leaf': 5, 'max_features': 0.2013010282596409}. Best is trial 1 with value: 0.1644008990458268.
[I 2024-12-09 16:30:36,135] Trial 3 finished with value: 1.7743850560388914 and parameters: {'n_e

Best parameters: {'n_estimators': 336, 'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 0.7768424530830121}


RandomForestRegressor(max_depth=26, max_features=0.7768424530830121,
                      min_samples_split=4, n_estimators=336, random_state=42)

In [15]:
y_hat = best_rf.predict(X_test)
r2 = r2_score(y_test, y_hat)
mae = mean_squared_error(y_test, y_hat)

print(f"R^2 Score: {r2:.4f}")
print(f"Mean Squared Error: {mae:.4f}")

R^2 Score: 0.9780
Mean Squared Error: 0.1483
